### Sednoid Daten 
Daten holen von https://ssd.jpl.nasa.gov/ und in wikitable konvertieren
Verwendung für [Wikipedia-Vorlage: Sednoiden](https://de.wikipedia.org/wiki/Vorlage:Sednoiden)

Libs importieren

In [ ]:
from urllib.request import urlopen
import json

JSON-Daten laden

In [ ]:
url='https://ssd-api.jpl.nasa.gov/sbdb_query.api?fields=epoch.cal,full_name,name,pdes,a,sigma_a,e,sigma_e,q,sigma_q,ad,sigma_ad,i,sigma_i,om,sigma_om,w,sigma_w,per.y,sigma_per,H,H_sigma,source&sb-kind=a&sb-cdata=%7B%22AND%22:%5B%22a%7CGE%7C150%22,%22q%7CGE%7C50%22%5D%7D'
with urlopen(url) as response:
    ssd = json.load(response)
    #print(ssd)


Objekt für jeden Asteroiden bauen

In [ ]:
sednoids = []
for asteroid in ssd['data']:
    # print(asteroid)
    sednoid = {}
    for field, val in zip(ssd['fields'], asteroid):
        sednoid[field] = val
        #print(field, val)
        
    sednoids.append(sednoid)

### Wikitable

Wikitable-Header und -Footer

In [ ]:
from string import Template
headertpl = Template("""{| class="wikitable sortable zebra" style="text-align:right; padding:2px 30px 2px 2px;"
|+ Bahnparameter hoch-extremer transneptunischer Objekte mit Perihelien größer 50 AE und großen Halbachsen """
"""weiter als 150 AE<ref name="Ssd-Api">{{Internetquelle 
    |url=https://ssd-api.jpl.nasa.gov/sbdb_query.api?fields=epoch.cal,full_name,name,pdes,a,sigma_a,e,sigma_e,q,sigma_q,ad,sigma_ad,i,sigma_i,om,sigma_om,w,sigma_w,per.y,sigma_per,H,H_sigma,source&sb-kind=a&sb-cdata=%7B%22AND%22:%5B%22a%7CGE%7C150%22,%22q%7CGE%7C50%22%5D%7D 
    |titel=SSD-Daten 
    |werk=[https://ssd.jpl.nasa.gov/tools/sbdb_query.html Small-Body Database Query] 
    |hrsg=Caltech/JPL 
    |format=JSON-Daten
    |datum=$epoch
    |abruf=$currentdate 
    |abruf-verborgen=1 }}</ref>
|- style="vertical-align:bottom;"
! Objekt
! [[Halbachsen der Ellipse|Halb&shy;achse]]<br />a (AE)
! [[Exzentrizität (Astronomie)|Exzen&shy;trizität]]<br />e
! [[Perihel]]<br />q (AE)
! [[Aphel]]<br />Q (AE)
! [[Bahnneigung|Inkli&shy;nation]]<br />i (°)
! [[Argument der Periapsis|Argument<br />der Periapsis]]<br />(°)
! [[Länge des aufsteigenden Knotens|Länge des aufst. Knotens]]<br />Ω (°)
! [[Umlaufzeit|Umlauf&shy;zeit]]<br />T (Jahre)
! [[Absolute Helligkeit]]<br />H (mag)
""")

footer="""|}<noinclude>

[[Kategorie:Vorlage:Astronomie]]
[[Kategorie:Vorlage:Tabelle]]
</noinclude>
"""

Hilfsfunktion um Toleranzen zu berechen, darzustellen und sie bei zu kleinen Werten auszublenden.

In [ ]:
def renderToleran(wert, sigma):
    if sigma * 100 / wert > 2:
        # Toleranz anzeigen
        return "{0:n} ± {1:.0f}".format(wert, sigma)
    else:
        # nur Wert
        return format(wert, "n")

Formatierung Name wikipedia-gerecht (einschließlich Wikilink)

In [ ]:
def wikiname(sed):
    if (sed['name'] == None):
        parts = sed['pdes'].split(' ')
        des = parts[1][:2]
        number = parts[1][2:]
        return '[[{}|{} {}<sub>{}</sub>]]'.format(sed['pdes'],parts[0], des, number)
    else:
        return '[[({}) {}]]'.format(sed['pdes'], sed['name'])

Über Asteroiden iterieren und Feldern aus den Asteroiden berechnen

In [ ]:
import locale
loc = locale.getlocale()
locale.setlocale(locale.LC_ALL, '')

out = []
for sed in sednoids:
    sed_out = {}
    for name,value in sed.items():
        if name in ['e','q','i','om','w','H']:
            sed_out[name] = format(float(sed[name]), "n")
            #print(name+': '+sed_out[name])
            
        elif name in ['a','ad']:
            sed_out[name] = renderToleran(float(sed[name]), float(sed['sigma_'+name]))
            #print(name+': '+sed_out[name])
            
        elif name == 'per.y':
            # sigma_per sind Tage
            sed_out[name] = renderToleran(float(sed[name]), float(sed['sigma_per'])/365)
            #print(name+': '+sed_out[name])
            
        elif name == 'full_name':
            sed_out[name] = wikiname(sed)
            #print(sed_out[name])
            
        else:
            sed_out[name] = str(sed[name])
            #print('unchanged '+sed_out[name])

    out.append(sed_out)
#print(out)

In [ ]:
from datetime import date
table = ""
for ast in out:
    #print(ast)
    # Start Zeile
    table += """|-
| style="text-align:left;" | """
    table += ast['full_name'].strip() + '\n'
    table += '| ' + ast['a']
    table += ' || ' + ast['e']
    table += ' || ' + ast['q']
    table += ' || ' + ast['ad']
    table += ' || ' + ast['i']
    table += ' || ' + ast['w']
    table += ' || ' + ast['om']
    table += ' || ' + ast['per.y']
    table += ' || ' + ast['H']

    table += '\n'
    
epoch=out[0]['epoch.cal'].split('.')[0]
header = headertpl.substitute(currentdate=date.today().isoformat(), epoch=epoch)
    
ergebnis = header + table + footer
print(ergebnis)